In [5]:
from core.utils import Tibanna
from core import ff_utils


# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)

file_not_found = 0
deleted = 0
uploading = 0
success = 0
patch_problem = 0
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)
files_resp = ff_utils.get_metadata('files', connection=ff)['@graph']
# sp_files_resp = [ff_utils.get_metadata('4DNFIC6AZHM5', connection=ff)]
# Search1 = '/search/?status=released&status=uploaded&status=released%20to%20project&type=File'
# Search2 = '/search/?lab.title=4DN%20DCIC%20Lab%2C%20HMS&type=FileFastq'

#add_size = ff_utils.get_metadata(Search1, connection=ff)['@graph']
#print len(add_size), 'files'

print len(files_resp)

no_size = 0
counter = 0
for a_file in files_resp:
    if a_file['status'] == 'deleted':
        deleted += 1
        continue
    counter += 1
    if counter % 100 == 0:
        print counter,"+" ,deleted
    if a_file['status'] in ['uploading', 'to be uploaded by workflow']:
        uploading += 1
        continue
    # check if there is  no filesize
    if not a_file.get('file_size'):
        # decide on the bucket
        if 'FileProcessed' in a_file['@type']:
            bucket = tibanna.s3.outfile_bucket
        else:
            bucket = tibanna.s3.raw_file_bucket
            
        # check if file is in s3
        head_info = tibanna.s3.does_key_exist(a_file['upload_key'], bucket)
        if not head_info:
            print a_file['@id'], a_file['status']
            file_not_found += 1
            continue
        file_size = head_info['ContentLength']

        patch_data = {'file_size': file_size}
        try:
            ff_utils.patch_metadata(patch_data, obj_id=a_file['uuid'] ,connection=ff)
            success += 1
        except Exception as e:
            print e
            print
            patch_problem += 1

print file_not_found, 'files missing from s3'
print patch_problem, 'files had patch problems'
print deleted, 'deleted files skipped'
print uploading, 'uploading files skipped'
print success, 'total files patched'

5584
100 + 0
200 + 0
300 + 0
400 + 0
500 + 0
600 + 0
700 + 0
800 + 0
900 + 0
1000 + 0
1100 + 0
1200 + 0
1300 + 0
1400 + 0
1500 + 0
1600 + 0
1700 + 0
1800 + 0
1900 + 0
2000 + 0
2100 + 0
2200 + 0
2300 + 0
2400 + 0
2500 + 0
2600 + 0
2700 + 0
2800 + 0
2900 + 0
3000 + 0
3100 + 0
3200 + 0
3300 + 0
3400 + 0
3500 + 0
3600 + 0
3700 + 0
3800 + 0
3900 + 0
4000 + 0
4100 + 0
4200 + 0
4300 + 0
4400 + 0
4500 + 0
4600 + 0
4700 + 0
4800 + 0
4900 + 0
5000 + 0
5100 + 0
5200 + 0
5300 + 0
5400 + 0
5500 + 0
0 files missing from s3
0 files had patch problems
0 deleted files skipped
305 uploading files skipped
7 total files patched
